### Setting directory

In [58]:
%cd C:\Users\Áron\Desktop

C:\Users\Áron\Desktop


### Imports


In [8]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
print(torch.__version__)

import warnings
warnings.filterwarnings('ignore')

1.5.0


In [9]:
dataset = torchvision.datasets.MNIST(
            root="./data/MNIST",
            train = True,
            download = True,
            transform = transforms.Compose([transforms.ToTensor()]))

### Buliding model

In [10]:
class Network(nn.Module):
    def __init__(self):
        super(Network,self).__init__()
        
        self.convolutional_1 = nn.Conv2d(in_channels = 1, out_channels = 6, kernel_size= 5)
        self.convolutional_2 = nn.Conv2d(in_channels = 6, out_channels = 12, kernel_size= 5)
        
        self.fully_connected_1 = nn.Linear(in_features = 12*4*4, out_features = 120)
        self.fully_connected_2 = nn.Linear(in_features = 120, out_features = 60)
        self.out = nn.Linear(in_features = 60, out_features = 10)
        
    def forward(self, n):
        n = self.convolutional_1(n)
        n = F.relu(n)
        n = F.max_pool2d(n, kernel_size = 2, stride = 2)
        
        n = self.convolutional_2(n)
        n = F.relu(n)
        n = F.max_pool2d(n, kernel_size = 2, stride = 2)
        
        n = n.reshape(-1, 12*4*4)
        n = self.fully_connected_1(n)
        n = F.relu(n)
        
        n= self.fully_connected_2(n)
        n = F.relu(n)
        
        n = self.out(n)
        
        
        return n

### Training 

In [12]:
network = Network()

batch_size = 100
learning_rate = 0.01
shuffle = True

def correct_predicted(predicted, labels):
    return predicted.argmax(dim = 1).eq(labels).sum().item()

dataset_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle = shuffle)
optimizer = optim.Adam(network.parameters(), lr = learning_rate)

images, labels = next(iter(dataset_loader))


tensor_board = SummaryWriter(comment = f"Batch size={batch_size} Learning rate={learning_rate} Shuffle={shuffle}")

for epoch in range(10):
    total_loss = 0
    total_correct = 0
    
    for batch in dataset_loader:
        images = batch[0]
        labels = batch[1]
        predicted = network(images)
        loss = F.cross_entropy(predicted, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        total_correct += correct_predicted(predicted, labels)
    
    tensor_board.add_scalar('Loss', total_loss, epoch)
    tensor_board.add_scalar('Correct Predictions', total_correct, epoch)
    tensor_board.add_scalar('Accuracy', total_correct / len(dataset), epoch)
    
    tensor_board.add_histogram('convolutional_1.bias', network.convolutional_1.bias, epoch)
    tensor_board.add_histogram('convolutional_1.weight', network.convolutional_1.weight, epoch)
    tensor_board.add_histogram('convolutional_1.weight.grad' ,network.convolutional_1.weight.grad, epoch)
    
    print(f"Epoch {epoch} Accuracy: {total_correct/len(dataset):.5f} loss: {total_loss:.4f}")
    
tensor_board.close()

Epoch 0 Accuracy: 0.93557 loss: 122.7565
Epoch 1 Accuracy: 0.97687 loss: 47.4656
Epoch 2 Accuracy: 0.98075 loss: 40.3921
Epoch 3 Accuracy: 0.98218 loss: 37.4653
Epoch 4 Accuracy: 0.98337 loss: 36.3379
Epoch 5 Accuracy: 0.98485 loss: 31.8181
Epoch 6 Accuracy: 0.98390 loss: 35.7959
Epoch 7 Accuracy: 0.98508 loss: 34.7383
Epoch 8 Accuracy: 0.98588 loss: 32.4139
Epoch 9 Accuracy: 0.98690 loss: 29.8898
